
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Dropout</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Dropout 
1. pojedyncze neurony uczą się zbyt dokładnie (mogą się uczyć)
  * overfitting
2. sieć tworzy czasem zbyt dużą współzależność grup neuronów
  * pary neuronów specjalizują się w rozpoznawaniu bardzo specyficznych problemów
  * dropout __losowo__ wyłącza neurony
  * neurony mają małą szansę na powtórzenie konfiguracji
3. pozwala na uczenie badziej zgrubnych cech
  * po wyłączeniu jedych, inne neurony muszą zastąpić tamte w rozpoznawaniu
  * można zastosować dla neuronów wejściowych
4. wprowadza efekt regularyzacji
  * dodawany jest szum do aktywacji
4. końcowy model jest __ensemblem__, przy czym liczba tworzonych modeli jest olbrzymia
  * w ensemblu końcowy model jest złożeniem wielu modeli podstawowych
  * każdy model rozwiązuje _prawie_ ten sam problem
  * uśrednienie daje model silniejszy
  * przy wyłączaniu tworzy się wiele niezaleźnych wewnętrznych reprezentacji
5. zwykle wymaga więcej epok uczenia
6. niekoniecznie dobrze współpracuje z BN

## Dropout - założenia
1. Nitish Srivastava, Geoffrey Hinton, Alex Krizhevsky, Ilya Sutskever, Ruslan Salakhutdinov
2. najlepszym sposobem, przy nieograniczonych zasobach, będzie
  * uśrednić predykcje po wszystkich ustawieniach parametrów
  * ważyć każde przez posterior prawdopodobieństwa przy zadanych danych
  * jest to zwykle możliwe tylko dla bardzo małych modeli
3. przy łączeniu modeli w ensemble ważne by modele były __różne__ (ang. diverse)
  * modele są różne, jeśli _popełniają błędy w różnych miejscach_
  * uczenie sieci jest kosztowne
4. Dropout stara się rozwiazać oba problemy
  * czasowo odrzucane są poszczególne neurony
    * widzialne lub ukryte
  * prawdopodobieństwo dla ukrytych jest zwykle około $0.5$
  * dla wejściowych nawet bliżej $1$!
  * sieć staje się bardzo rozrzedzona
  * wybór neuronów można zaimplementować jako przemnożenie wszystkich aktywacji przez binarną maskę
    * łatwe w implementacji dla współczesnych procesorów
    * wygodne także w algorytmie SGD
5. poszczególne neurony __nie mogą__ się teraz uczyć skomplikowanych wzorców
  * niektóre wejścia są czasem aktywne, czasem nie
  * to generalizuje
6. uczenie mioże być wolniejsze
5. pełna sieć to zbiór $2^n$ różnych zubożonych modeli
  * dzielą jednak parametry
  * połączenie słabych modeli może dawać model silny

### Dropout jako regularyzacja
7. Dropout jest metodą regularyzacji przez dodanie szumu
  * tzw. _denoising_ autoencoder dostaje zaszumione dane
  * model ma odtworzyć dane niezaszumione

## Dropout
1. sieć warstwowa
$$\begin{align}
\textbf{bez dropout}\hskip{2em}&\hskip{2em}\textbf{z dropout}\\
\hskip{2em}&\hskip{2em}r\sim{}Bernoulli(p)\\
\hskip{2em}&\hskip{2em}\widetilde{y}^{(l)}=r*y^{(l)}\\
z_i^{(l)}=w_i^{(l+1)}\,y^{(l)}+b_i^{(l+1)}\hskip{2em}&\hskip{2em}z_i^{(l)}=w_i^{(l+1)}\,\widetilde{y}^{(l)}+b_i^{(l+1)}\\
y_i^{(l+1)}=g(z_i^{(l+1)})\hskip{2em}&\hskip{2em}y_i^{(l+1)}=g(z_i^{(l+1)})
\end{align}$$
2. uczenie
  * dowolny algorytm typu SGD
  * po wylosowaniu neuronów wsteczna propagacja __wyłącznie__ dla pozostawionych
  * ważną formą regularyzacji jest __ograniczenie__ normy wektora wag wchodzącego do dowolnego neurona do ustalonej wartości
    * tzw. __max-norm__ przez ograniczenie wektora do kuli o promieniu $c$
  * także wysokie wartości __decay__ przyspieszają zbieżność

### Inverted dropout
* wyzerowanie neuronów w warstwie z prawdopodobieństwem $p$ redukuje wartość oczekiwaną wejść nauronów następnej warstwy o $1-p$
* po wyzerowaniu, przemnożyć wartości warstwy liniowej przez $1-p$
* sprawdza się lepiej podczas testowania

## Ewaluacja w czasie testowania
1. __nie__ używać dropout w czasie testowania
  * wyjście nie ma być teraz stochastyczne
2. obliczenia z pomocą modelu dropout
  * wykonać wiele powtórzeń stosując dropout
  * wystarczająco dużo obliczeń da dobre wyniki
  * pracochłonne
3. model obliczony wprost ma wadę, bo wartości oczekiwane aktywacji różnią się od prawdziwych
  * inverted dropout niweluje ten efekt już w trakcie uczenia
4. bez inverted dropout
  * przemnożyć aktywacje przez wartość $1-p$
  * wybranie modelu i przemnożenie wag

## Czemu dropout działa
1. usunięcia silnych wzorców z uczenia
2. podobne do L2 w procesie osłabiania nieistotnych wag
  * można pokazać, że czyni to nawet bardziej inteligentnie
  * ensemble
    * wybrać $k$ modeli
      * dla każdego postąpić znaleźć predykcje
      * wszystkie wyniki uśrednić
      * dla MNIST ok. $50$ modeli równa się z pełnym mnożonym
      * zwykle wystarczy wziąć ok. $10$ niezależnych predykcji
    * bardziej kosztowne
4. Cechy uczenia
  * uczenie wymaga zwykle sporo większej liczby epok
  * neurony ukryte mają rzadką aktywację
  * parametr prawdopodobieństwa pozostawienie neuronu $p$ można wybrać
    * niskie $p$ (mało neuronów) powoduje niedofitowanie,  (dla MNIST)
    * zwykle $p\geq0.4$ daje najlepsze wyniki

## Uwagi
* często parametr $p$ jest ustalany osobno dla warstw
  * im więcej jest wag między warstwami (duże warstwy), tym mniejsze prawdopodobieństwo usunięcia $p$ (dla neuronów warstwy wcześniejszej) 
* można także zastosować do neuronów wejściowych
* bardzo często dla przetwarzania obrazów
  głównie ze wzgledu na brak danych
* funkcja kosztu jest stochastyczna i nie dość dobrze zdefiniowana
  * debugować bez włączonego dropoutu